# Long-Duration Spotify Tracks (2014–2024): Data Cleaning & QA

This notebook cleans and validates a dataset of 816 long-duration Spotify tracks collected via the Spotify public API.

**Goals**
- Standardize column names and data types
- Check for duplicates and missing values
- Clean track names and artist fields (split multi-artist strings)
- Generate basic quality checks and summaries
- Export a cleaned dataset for a Streamlit dashboard

In [8]:
import pandas as pd
import numpy as np
import re

pd.set_option("display.max_colwidth", 120)

df_raw = pd.read_csv("data.csv")
df = df_raw.copy()
df.head()

,ID,Name,Duration (Minutes),Artists
0,026O2YP2wRl9h3Ht9ClZST,Steady Rain in a Forest with Light Background Bird Chirping (Storm Ambience Sound Effects),100,Nature Sounds
1,0DXnjJxJBvgMvBjG6Jug7I,Soundarya Lahari,66,Mambalam Sisters
2,6BFn78qFy9bRqKHTMfhnNv,Waves of Abundance & Fullfillment,60,Zen Life Relax
3,1dNwMOMtq3yflXibbqgLku,Mosquito Solution Anti Teen Street Party or Teenagers Loitering,45,Device Sound Effect
4,290gfei8moWWSh8qTVkTT3,Ibiza Sunset Tibetan Singing Bowl Sessions 7 (Wipe out All Negativity Inside You),44,Tibetan Singing Bowls


## Quick Snapshot
We start by checking shape, dtypes, missing values, and basic statistics.

In [9]:
print("Shape:", df.shape)
display(df.head(10))
df.info()
df.describe(include="all")

Shape: (816, 4)


,ID,Name,Duration (Minutes),Artists
0,026O2YP2wRl9h3Ht9ClZST,Steady Rain in a Forest with Light Background Bird Chirping (Storm Ambience Sound Effects),100,Nature Sounds
1,0DXnjJxJBvgMvBjG6Jug7I,Soundarya Lahari,66,Mambalam Sisters
2,6BFn78qFy9bRqKHTMfhnNv,Waves of Abundance & Fullfillment,60,Zen Life Relax
3,1dNwMOMtq3yflXibbqgLku,Mosquito Solution Anti Teen Street Party or Teenagers Loitering,45,Device Sound Effect
4,290gfei8moWWSh8qTVkTT3,Ibiza Sunset Tibetan Singing Bowl Sessions 7 (Wipe out All Negativity Inside You),44,Tibetan Singing Bowls
5,23ordmf2KkRfHh9zE9RVyq,佃祭,35,Gontaro Yanagiya
6,47iSvlaCBugj3HrGZDZYcO,Tālā Gaisma (Distant Light),33,"Pēteris Vasks, O/Modernt Chamber Orchestra, Hugo Ticciati"
7,6fKZZf5Mws3hr31ZAvCOcs,Asmr Tea Massage Roleplay,32,Jessica Cross
8,4J08TVw9zNYeBUtSDIkLP6,Snuggling Up,31,Relaxmydog
9,1IPbXZ2OblHvSUY3Mj58iv,"Keep Driving, I’m Dreaming",30,"Nicole Lizée, Canada's National Arts Centre Orchestra, Alexander Shelley"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  816 non-null    object
 1   Name                816 non-null    object
 2   Duration (Minutes)  816 non-null    int64 
 3   Artists             816 non-null    object
dtypes: int64(1), object(3)
memory usage: 25.6+ KB


,ID,Name,Duration (Minutes),Artists
count,816,816,816.000000,816
unique,816,816,NaN,812
top,026O2YP2wRl9h3Ht9ClZST,Steady Rain in a Forest with Light Background Bird Chirping (Storm Ambience Sound Effects),NaN,Suara
freq,1,1,NaN,2
mean,NaN,NaN,6.980392,NaN
std,NaN,NaN,5.907642,NaN
min,NaN,NaN,5.000000,NaN
25%,NaN,NaN,5.000000,NaN
50%,NaN,NaN,6.000000,NaN
75%,NaN,NaN,7.000000,NaN


## Standardize Column Names
We’ll rename columns to snake_case and remove special characters for consistency in downstream work.

In [10]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(r"[^\w\s]", "", regex=True)
      .str.replace(r"\s+", "_", regex=True)
)

df.columns

Index(['id', 'name', 'duration_minutes', 'artists'], dtype='object')

## Data Quality Checks
- Missing values
- Duplicate track IDs
- Duplicate rows
- Unusual durations

In [11]:
# Missing values
missing = df.isna().sum().sort_values(ascending=False)
display(missing)

# Duplicate IDs
dup_id_count = df["id"].duplicated().sum()
print("Duplicate IDs:", dup_id_count)

# Duplicate full rows
dup_row_count = df.duplicated().sum()
print("Duplicate rows:", dup_row_count)

# Duration sanity checks
print("Duration min/max:", df["duration_minutes"].min(), df["duration_minutes"].max())
display(df["duration_minutes"].describe())

id                  0
name                0
duration_minutes    0
artists             0
dtype: int64

Duplicate IDs: 0
Duplicate rows: 0
Duration min/max: 5 100


count    816.000000
mean       6.980392
std        5.907642
min        5.000000
25%        5.000000
50%        6.000000
75%        7.000000
max      100.000000
Name: duration_minutes, dtype: float64

## Fix Data Types and Whitespace
- Ensure duration is numeric
- Strip whitespace in text columns
- Keep IDs as strings

In [12]:
# Strip whitespace for text columns
for col in ["id", "name", "artists"]:
    df[col] = df[col].astype(str).str.strip()

# Ensure numeric duration
df["duration_minutes"] = pd.to_numeric(df["duration_minutes"], errors="coerce")

# Re-check
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                816 non-null    object
 1   name              816 non-null    object
 2   duration_minutes  816 non-null    int64 
 3   artists           816 non-null    object
dtypes: int64(1), object(3)
memory usage: 25.6+ KB


id                  0
name                0
duration_minutes    0
artists             0
dtype: int64

## Clean Track Names
We’ll do light cleaning suitable for analysis:
- Remove excessive whitespace
- Normalize weird punctuation spacing
- Keep original casing (Spotify titles are meaningful)

In [13]:
def clean_title(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    s = re.sub(r"\s+([,.;:!?])", r"\1", s)  # remove spaces before punctuation
    return s

df["name_clean"] = df["name"].apply(clean_title)

# Show examples where it changed
changed = df[df["name"] != df["name_clean"]][["name", "name_clean"]].head(10)
display(changed)


,name,name_clean
701,怒りの矢を放て !,怒りの矢を放て!
804,"One more time , One more chance","One more time, One more chance"


## Parse Artists
The `artists` column may include multiple artists in a single string.  
We’ll split into a list, then create a primary artist for grouping, while preserving the full list.

In [14]:
def split_artists(s: str) -> list:
    """
    Attempts to split artist strings on common separators.
    You can expand this if your dataset uses other separators.
    """
    s = s.strip()
    # common separators: comma, ' & ', ' and ', ';'
    parts = re.split(r"\s*,\s*|\s*&\s*|\s+and\s+|;\s*", s)
    parts = [p.strip() for p in parts if p.strip()]
    return parts

df["artist_list"] = df["artists"].apply(split_artists)
df["artist_count"] = df["artist_list"].apply(len)
df["primary_artist"] = df["artist_list"].apply(lambda x: x[0] if len(x) else np.nan)

df[["artists", "artist_list", "artist_count", "primary_artist"]].head(10)

,artists,artist_list,artist_count,primary_artist
0,Nature Sounds,[Nature Sounds],1,Nature Sounds
1,Mambalam Sisters,[Mambalam Sisters],1,Mambalam Sisters
2,Zen Life Relax,[Zen Life Relax],1,Zen Life Relax
3,Device Sound Effect,[Device Sound Effect],1,Device Sound Effect
4,Tibetan Singing Bowls,[Tibetan Singing Bowls],1,Tibetan Singing Bowls
5,Gontaro Yanagiya,[Gontaro Yanagiya],1,Gontaro Yanagiya
6,"Pēteris Vasks, O/Modernt Chamber Orchestra, Hugo Ticciati","[Pēteris Vasks, O/Modernt Chamber Orchestra, Hugo Ticciati]",3,Pēteris Vasks
7,Jessica Cross,[Jessica Cross],1,Jessica Cross
8,Relaxmydog,[Relaxmydog],1,Relaxmydog
9,"Nicole Lizée, Canada's National Arts Centre Orchestra, Alexander Shelley","[Nicole Lizée, Canada's National Arts Centre Orchestra, Alexander Shelley]",3,Nicole Lizée


## Handle Duplicates
If duplicate track IDs exist, we keep the first occurrence (or you can pick the max duration).

In [15]:
if df["id"].duplicated().any():
    before = len(df)
    df = df.drop_duplicates(subset=["id"], keep="first").reset_index(drop=True)
    after = len(df)
    print(f"Dropped {before-after} duplicate IDs.")
else:
    print("No duplicate IDs found.")

No duplicate IDs found.


## Duration Range Checks
This dataset is supposed to contain “long-duration” tracks.  
We’ll flag anything < 5 minutes or suspiciously high (e.g., > 180).

In [16]:
flag_short = df[df["duration_minutes"] < 5]
flag_long = df[df["duration_minutes"] > 180]

print("Tracks < 5 minutes:", len(flag_short))
print("Tracks > 180 minutes:", len(flag_long))

display(flag_short.head(10))
display(flag_long.head(10))

Tracks < 5 minutes: 0
Tracks > 180 minutes: 0


,id,name,duration_minutes,artists,name_clean,artist_list,artist_count,primary_artist


,id,name,duration_minutes,artists,name_clean,artist_list,artist_count,primary_artist


## Final Dataset + Export
We’ll keep:
- `id`
- `name_clean`
- `duration_minutes`
- `artists` (original)
- `artist_list`
- `primary_artist`
- `artist_count`

Then export for the Streamlit dashboard.

In [17]:
df_clean = df[[
    "id",
    "name_clean",
    "duration_minutes",
    "artists",
    "artist_list",
    "primary_artist",
    "artist_count"
]].copy()

df_clean = df_clean.sort_values("duration_minutes", ascending=False).reset_index(drop=True)

display(df_clean.head(10))
df_clean.to_csv("data_clean.csv", index=False)
print("Saved: data_clean.csv")

,id,name_clean,duration_minutes,artists,artist_list,primary_artist,artist_count
0,026O2YP2wRl9h3Ht9ClZST,Steady Rain in a Forest with Light Background Bird Chirping (Storm Ambience Sound Effects),100,Nature Sounds,[Nature Sounds],Nature Sounds,1
1,0DXnjJxJBvgMvBjG6Jug7I,Soundarya Lahari,66,Mambalam Sisters,[Mambalam Sisters],Mambalam Sisters,1
2,6BFn78qFy9bRqKHTMfhnNv,Waves of Abundance & Fullfillment,60,Zen Life Relax,[Zen Life Relax],Zen Life Relax,1
3,1dNwMOMtq3yflXibbqgLku,Mosquito Solution Anti Teen Street Party or Teenagers Loitering,45,Device Sound Effect,[Device Sound Effect],Device Sound Effect,1
4,290gfei8moWWSh8qTVkTT3,Ibiza Sunset Tibetan Singing Bowl Sessions 7 (Wipe out All Negativity Inside You),44,Tibetan Singing Bowls,[Tibetan Singing Bowls],Tibetan Singing Bowls,1
5,23ordmf2KkRfHh9zE9RVyq,佃祭,35,Gontaro Yanagiya,[Gontaro Yanagiya],Gontaro Yanagiya,1
6,47iSvlaCBugj3HrGZDZYcO,Tālā Gaisma (Distant Light),33,"Pēteris Vasks, O/Modernt Chamber Orchestra, Hugo Ticciati","[Pēteris Vasks, O/Modernt Chamber Orchestra, Hugo Ticciati]",Pēteris Vasks,3
7,6fKZZf5Mws3hr31ZAvCOcs,Asmr Tea Massage Roleplay,32,Jessica Cross,[Jessica Cross],Jessica Cross,1
8,4J08TVw9zNYeBUtSDIkLP6,Snuggling Up,31,Relaxmydog,[Relaxmydog],Relaxmydog,1
9,1IPbXZ2OblHvSUY3Mj58iv,"Keep Driving, I’m Dreaming",30,"Nicole Lizée, Canada's National Arts Centre Orchestra, Alexander Shelley","[Nicole Lizée, Canada's National Arts Centre Orchestra, Alexander Shelley]",Nicole Lizée,3


Saved: data_clean.csv


## QA Summary (for Portfolio)
This section provides quick insights and confirms the dataset is analysis-ready.

In [18]:
qa = {
    "rows_clean": len(df_clean),
    "unique_tracks": df_clean["id"].nunique(),
    "unique_primary_artists": df_clean["primary_artist"].nunique(),
    "min_duration": float(df_clean["duration_minutes"].min()),
    "median_duration": float(df_clean["duration_minutes"].median()),
    "max_duration": float(df_clean["duration_minutes"].max()),
    "multi_artist_tracks": int((df_clean["artist_count"] > 1).sum()),
}

qa_df = pd.DataFrame([qa])
display(qa_df)

top_artists = df_clean["primary_artist"].value_counts().head(15).reset_index()
top_artists.columns = ["primary_artist", "track_count"]
display(top_artists)

,rows_clean,unique_tracks,unique_primary_artists,min_duration,median_duration,max_duration,multi_artist_tracks
0,816,816,796,5.0,6.0,100.0,236


,primary_artist,track_count
0,Ludwig van Beethoven,5
1,Antonín Dvořák,3
2,Wolfgang Amadeus Mozart,3
3,Traditional,3
4,Pyotr Ilyich Tchaikovsky,2
5,Suara,2
6,Anton Bruckner,2
7,Harris Jayaraj,2
8,Johann Sebastian Bach,2
9,Nusrat Fateh Ali Khan,2
